# LoneSTAR

Baseline from:

I. P. Roberts, S. Vishwanath, and J. G. Andrews, “LoneSTAR: Analog Beamforming Codebooks for Full-Duplex Millimeter Wave Systems,” IEEE Transactions on Wireless Communications, vol. 22, no. 9, pp. 5754–5769, Sep. 2023, [doi: 10.1109/TWC.2023.3236352](https://doi.org/10.1109/TWC.2023.3236352).


In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import cvxpy as cp

import project_path as pp
from cissir import params, optimization as opt
from cissir.physics import pow2db, db2power, mag2db
from cissir.beamforming import dft_codebook
from cissir.utils import PrintBuffer

base_path = pp.module_path
plot_path = base_path/"plots"
res_path = base_path/"results"

In [2]:
# Global parameters

c = params.c
fc = params.fc_hz
wavelength_m = params.wavelength_m

ula_el_spacing = params.array_electrical_spacing

N_r = params.n_rx
N_t = params.n_tx

# Number of beams
L_r = params.num_beams
L_t = params.num_beams

### DFT beam codebook


In [3]:
dft_tx_codebook, tx_degs = dft_codebook(L_max = L_t , N1 = N_t,
                                          az_min = -60, az_max = 60, transmit=True)
dft_rx_codebook, rx_degs = dft_codebook(L_max = L_r , N1 = N_r,
                                          az_min = -60, az_max = 60, transmit=False)

In [4]:
si_num_taps = 1

with np.load(res_path/"channel_impulse_responses.npz") as rt_data:
    t_rt = np.squeeze(rt_data['t_channel_s'])
    ht_si_full = np.squeeze(rt_data['ht_si'])
    h_full = np.sum(ht_si_full, axis=0).transpose(2, 0, 1)

with np.load(res_path/"si_mimo.npz") as rt_data:
    h_si = rt_data['h_si_matrix'][:si_num_taps]
    
    h_cir = np.sum(ht_si_full[:si_num_taps,], axis=0).transpose(2, 0, 1)
    si_pow_db = mag2db(np.max(opt.codebook_si(dft_tx_codebook, dft_rx_codebook, h_cir)))

    assert h_si.shape == (si_num_taps, N_r, N_t), "Unexpected SI shape"

h_si_unnormalized = np.squeeze(h_si)


In [5]:
si_norm = np.linalg.norm(h_si_unnormalized)

# ||H||_F=1.0
h_si = h_si_unnormalized/si_norm

si_normalized_ref = mag2db(np.max(opt.codebook_si(dft_tx_codebook, dft_rx_codebook, h_si)))
si_norm_db = float(si_pow_db) - si_normalized_ref
si_db_ref = si_normalized_ref + si_norm_db

In [10]:
# Prob (43)

phased_array = False
max_att = -100

def feasible_codebook(codebook, equal_amp=phased_array):
    n_ant = codebook.shape[0]
    if equal_amp:
        feas_cb = codebook/np.abs(codebook)
    else:
        feas_cb = codebook * np.sqrt(n_ant)/np.linalg.norm(codebook, axis=0, keepdims=True)
    return feas_cb

A_tx = cp.Constant(dft_tx_codebook/np.abs(dft_tx_codebook))
A_rx = cp.Constant(dft_rx_codebook/np.abs(dft_rx_codebook))

error_db_values = np.arange(-80, -2, 1)

results = []
tx_codebooks = []
rx_codebooks = []
pbuf = PrintBuffer(print_input=False)
for error_db in tqdm(error_db_values):

    threshold_tx = db2power(error_db) * L_t * N_t ** 2
    
    W = cp.Constant(dft_rx_codebook)
    F = cp.Variable(shape=(N_t, L_t), complex=True)
    F.value = dft_tx_codebook
    
    obj_tx = cp.Minimize(cp.norm(cp.diag(W.H @ h_si @ F), p=2)**2)
    similarity_tx = cp.norm(N_t * np.ones(L_t) - cp.diag(A_tx.H @ F), p=2)**2 <= threshold_tx
    feasible_tx = cp.abs(F) <= cp.Constant(1.0)
    
    problem_tx = cp.Problem(obj_tx, constraints=[similarity_tx, feasible_tx])
    problem_tx.solve(solver='SCS', warm_start=True)
    
    tx_status = problem_tx.status
    if tx_status != 'optimal':
        pbuf.print(f"Tx error for {error_db} dB: {tx_status}")
    
    F_sol = feasible_codebook(F.value) # Make realizable
    threshold_rx = db2power(error_db) * L_r * N_r ** 2
    
    W = cp.Variable(shape=(N_r, L_r), complex=True)
    F = cp.Constant(F_sol)
    W.value = dft_rx_codebook
    
    obj_rx = cp.Minimize(cp.norm(W.H @ h_si @ F, p='fro')**2)
    
    similarity_rx = cp.norm(N_r * np.ones(L_r) - cp.diag(A_rx.H @ W), p=2)**2 <= threshold_rx
    feasible_rx = cp.abs(W) <= cp.Constant(1.0)
    
    problem_rx = cp.Problem(obj_rx, constraints=[similarity_rx, feasible_rx])
    problem_rx.solve(solver='SCS', warm_start=True)

    
    rx_status = problem_rx.status
    if rx_status != 'optimal':
        pbuf.print(f"Rx error for {error_db} dB: {rx_status}")
    
    W_sol = feasible_codebook(W.value) # Make realizable

    f_codebook = F_sol/np.sqrt(N_t)
    w_codebook = W_sol/np.sqrt(N_r)
    
    si_db_1tap = mag2db(np.max(opt.codebook_si(f_codebook, w_codebook, h_si)))
    si_db_full = mag2db(np.max(opt.codebook_si(f_codebook, w_codebook, h_full)))

    att_db = -(si_norm_db+si_db_1tap)
    att_full_db = -si_db_full

    if att_db > max_att:
        max_att = att_db
    elif att_db < max_att -2.0:
        print("SI attenuation does not improve. Exiting loop...")
        break

    error_f = pow2db(max(opt.codebook_deviation_power(dft_tx_codebook, f_codebook), 1e-10))
    error_w = pow2db(max(opt.codebook_deviation_power(dft_rx_codebook, w_codebook), 1e-10))
    
    results.append({"loss_tgt": error_db, "opt": "lonestar",
                    "att_opt": att_db, "tx_loss": error_f, "rx_loss": error_w,
                    "att_full": att_full_db,
                    "phased": phased_array, "si_taps": si_num_taps, "num_beams": L_t})
    
    pbuf.print(f"Optimization for error {error_db:.1f} dB")
    pbuf.print(f"\tTx error:\t{error_f:.1f} dB")
    pbuf.print(f"\tRx error:\t{error_w:.1f} dB")
    pbuf.print(f"\tSI att 1-tap:\t{att_db:.1f} dB")
    pbuf.print(f"\tFull SI att:\t{att_full_db:.1f} dB")
    pbuf.print("\n")
    
    tx_codebooks.append(f_codebook)
    rx_codebooks.append(w_codebook)


  0%|          | 0/78 [00:00<?, ?it/s]

SI attenuation does not improve. Exiting loop...


In [11]:
pbuf.print()

Tx error for -80 dB: optimal_inaccurate
Rx error for -80 dB: optimal_inaccurate
Optimization for error -80.0 dB
	Tx error:	-31.2 dB
	Rx error:	-34.0 dB
	SI att 1-tap:	70.3 dB
	Full SI att:	69.1 dB


Tx error for -79 dB: optimal_inaccurate
Rx error for -79 dB: optimal_inaccurate
Optimization for error -79.0 dB
	Tx error:	-31.2 dB
	Rx error:	-33.9 dB
	SI att 1-tap:	70.3 dB
	Full SI att:	69.1 dB


Tx error for -78 dB: optimal_inaccurate
Rx error for -78 dB: optimal_inaccurate
Optimization for error -78.0 dB
	Tx error:	-31.2 dB
	Rx error:	-33.8 dB
	SI att 1-tap:	70.3 dB
	Full SI att:	69.1 dB


Tx error for -77 dB: optimal_inaccurate
Rx error for -77 dB: optimal_inaccurate
Optimization for error -77.0 dB
	Tx error:	-31.1 dB
	Rx error:	-33.7 dB
	SI att 1-tap:	70.4 dB
	Full SI att:	69.1 dB


Tx error for -76 dB: optimal_inaccurate
Rx error for -76 dB: optimal_inaccurate
Optimization for error -76.0 dB
	Tx error:	-31.1 dB
	Rx error:	-33.5 dB
	SI att 1-tap:	70.4 dB
	Full SI att:	69.1 dB


Tx er

## Saving results

In [12]:
suffix = "phased" if phased_array else "taper"
res_df = pd.DataFrame(results)
res_df.to_csv(res_path/f"lonestar_cb_results_{suffix}.csv")

In [9]:
error_array = np.insert(error_db_values.astype(float), 0, -np.inf)
tx_codebooks = np.stack([dft_tx_codebook] + tx_codebooks, axis=0)
rx_codebooks = np.stack([dft_rx_codebook] + rx_codebooks, axis=0)

metadata = np.array({"method": "lonestar", "phased": phased_array, "si_num_taps": si_num_taps})
fname = f"lonestar_codebooks.npz"
np.savez(res_path/fname, error=error_array,
         rx_degrees=rx_degs, tx_degrees=tx_degs,
         rx=rx_codebooks, tx=tx_codebooks,
         metadata=metadata)
print(f"{fname} was saved at {res_path}.")

lonestar_codebooks.npz was saved at /mnt/project/results.
